<a href="https://colab.research.google.com/github/hanadoolsae/ewhaguidebook/blob/main/data/bard/book_difficulty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [ ]:
!pip install bardapi
!pip install transformers
!pip install konlpy


## **BARD 이용해서 엑셀 돌리기**

In [ ]:
import os
import bardapi
import re
import openpyxl
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from bardapi import Bard
import time

token='Secure-1PSID Value'

In [ ]:
# 엑셀 파일 로딩
workbook = openpyxl.load_workbook("/content/data.xlsx")
sheet = workbook.active# 필요한 열 추출
data = pd.DataFrame(sheet.values)

book_data = data.iloc[1:, [0, 1, 2, 3, 4, 5, 6]]
book_data.columns = ["book_isbn", "book_title", "book_author", "book_publisher", "book_date", "genre_names", "book_genres"]

In [ ]:
# 새로운 엑셀 파일 생성
new_workbook = openpyxl.Workbook()
new_sheet = new_workbook.active
new_sheet.append(["book_isbn", "book_title", "book_author", "book_publisher", "book_date", "genre_names", "book_genres", "book_difficulty"])


# **난이도 등급 추출**

In [ ]:
def get_book_difficulty(book_isbn, book_title, book_author, book_publisher, book_date, genre_names, book_genres):

    # 프롬프트 작성
    book_info = f"Book: {book_title} by {book_author}, Category: {genre_names}, Publisher: {book_publisher}"
    prompt_with_book_info = (
        "이 Excel 파일에는 다음과 같은 정보가 포함되어 있습니다: 책 isbn, 책 제목, 책 작가, 책 출판사, 책 발간년도, 책의 장르, 책의 장르번호.\n"
        f"이 Excel 파일에서 각 행을 읽고 정보를 사용하여 책의 난이도 등급을 판별하고 1에서 5까지의 난이도 등급을 할당하십시오.\n"
        f"등급은 책의 길이, 내용 복잡성, 사용된 어휘 수준, 작가의 문체 및 주제의식이나 장르와 같은 다양한 요소를 고려하여 결정해야 합니다. \n"
        f"난이도 1은 초등학생이 읽을 수 있을 수준, 난이도 3은 중학교 3학년에서 고등학교 1학년이 읽을 수준, 난이도 5는 성인이 읽기에도 어려울 수 있는 수준입니다."
        f"왜 이러한 난이도로 산정하였는지 이유를 3줄 이내로 작성해야합니다."
        + book_info
    )

    # Bard API에서 텍스트 생성
    response = bardapi.core.Bard(token).get_answer(prompt_with_book_info)

    # 생성된 텍스트에서 난이도 등급 추출
    book_difficulty = response['content'].strip()

    return book_difficulty



In [ ]:
batch_size = 100
for i in range(0, len(book_data), batch_size):  # len(book_data) 추가
    batch = book_data[i:i+batch_size]
    for _, row in batch.iterrows():
        book_isbn, book_title, book_author, book_publisher, book_date, genre_names, book_genres = row["book_isbn"], row["book_title"], row["book_author"], row["book_publisher"], row["book_date"], row["genre_names"], row["book_genres"]

        # 모든 열이 비어 있는 경우 빈 행으로 처리
        if pd.isnull(book_isbn) and pd.isnull(book_title) and pd.isnull(book_author) and pd.isnull(book_publisher) and pd.isnull(book_date) and pd.isnull(genre_names) and pd.isnull(book_genres):
            continue

        book_difficulty = get_book_difficulty(book_isbn, book_title, book_author, book_publisher, book_date, genre_names, book_genres)
        new_sheet.append([book_isbn, book_title, book_author, book_publisher, book_date, genre_names, book_genres, book_difficulty])

    # 20초 쉬기
    time.sleep(20)  # 20초로 변경


# **엑셀 저장**

In [ ]:
# 새로운 엑셀 파일 저장
new_workbook.save("result.xlsx")